# Treinamento do Modelo de OCREste notebook é dedicado ao treinamento de uma Rede Neural Convolucional (CNN) para a classificação de caracteres (visografemas) do dataset sintético gerado anteriormente. As principais etapas são:- **Carregamento dos Dados**: Carrega os conjuntos de treino, validação e teste a partir da estrutura de diretórios criada.- **Construção do Modelo**: Define a arquitetura da CNN, incluindo camadas convolucionais, de pooling, dropout e de classificação.- **Treinamento**: Compila e treina o modelo utilizando os dados de treino e validação, monitorando a acurácia e a perda ao longo das épocas.- **Avaliação**: Avalia o desempenho do modelo treinado no conjunto de teste para verificar sua capacidade de generalização.- **Visualização**: Plota gráficos da acurácia e perda durante o treinamento para análise do comportamento do modelo.

## Importação de BibliotecasImporta as bibliotecas essenciais para o treinamento do modelo, incluindo TensorFlow para a construção e treinamento da rede neural, Matplotlib para visualização e `os` e `pathlib` para manipulação de caminhos de arquivo.

In [5]:
import tensorflow as tffrom tensorflow.keras import layers, modelsimport matplotlib.pyplot as pltimport numpy as npimport osfrom pathlib import Path

## Localização da Raiz do ProjetoA função `find_project_root` localiza o diretório raiz do projeto para garantir que os caminhos para os dados sejam resolvidos corretamente.

In [6]:
def find_project_root(markers=(pyproject.toml, .git)) -> Path:    '''    Descobre a raiz do projeto subindo diretórios até encontrar um marcador.    Parâmetros:        markers (tuple): Nomes de arquivos/pastas que indicam a raiz (ex.: pyproject.toml, .git).    Retorna:        pathlib.Path: Aponta para a raiz do projeto, ou o diretório atual como fallback.    '''    p = Path.cwd().resolve()    for parent in [p, *p.parents]:        if any((parent / m).exists() for m in markers):            return parent    return pROOT = find_project_root()

## Configurações do TreinamentoDefine os hiperparâmetros e configurações para o processo de treinamento, incluindo caminhos, dimensões das imagens, tamanho do lote (batch size), número de épocas e o número total de classes.

In [11]:
# --- Configurações ---# ConfiguraçõesDATA_DIR = ROOT / dataDATASET_DIR = DATA_DIR / raw / datasetIMG_HEIGHT = 64IMG_WIDTH = 64BATCH_SIZE = 32EPOCHS = 20NUM_CLASSES = 149 # Número de símbolosMODEL_SAVE_PATH = DATA_DIR / processed / modelo_ocr_simbolos.keras

## Carregamento e Preparação dos DadosA função `load_datasets` utiliza a utilidade `image_dataset_from_directory` do TensorFlow para carregar os dados de treino, validação e teste diretamente dos diretórios. A função garante que as classes sejam mapeadas de forma consistente entre os conjuntos.

In [8]:
# --- Carregamento e Preparação dos Dados ---def load_datasets():    '''Carrega os datasets de treino, validação e teste a partir dos diretórios.    Utiliza `tf.keras.preprocessing.image_dataset_from_directory` para criar lotes    de imagens e rótulos. A função também garante que os nomes das classes (e, portanto,    os índices) sejam consistentes entre os diferentes conjuntos de dados.    Retorna:        tuple: Uma tupla contendo (train_ds, val_ds, test_ds, class_names).    '''    class_names = sorted(os.listdir(os.path.join(DATASET_DIR, 'train')))    train_ds = tf.keras.preprocessing.image_dataset_from_directory(        os.path.join(DATASET_DIR, 'train'),        image_size=(IMG_HEIGHT, IMG_WIDTH),        batch_size=BATCH_SIZE,        color_mode='grayscale',        # lambel_mode='categorical',        label_mode='int',        validation_split=None,        # seed=123,        class_names=class_names  # garante mapeamento idêntico    )    val_ds = tf.keras.preprocessing.image_dataset_from_directory(        os.path.join(DATASET_DIR, 'validation'),        image_size=(IMG_HEIGHT, IMG_WIDTH),        batch_size=BATCH_SIZE,        color_mode='grayscale',        # label_mode='categorical',        label_mode='int',        validation_split=None,        # seed=123,        class_names=class_names  # garante mapeamento idêntico    )    test_ds = tf.keras.preprocessing.image_dataset_from_directory(        os.path.join(DATASET_DIR, 'test'),        image_size=(IMG_HEIGHT, IMG_WIDTH),        batch_size=BATCH_SIZE,        color_mode='grayscale',        # label_mode='categorical',        label_mode='int',        class_names=class_names    )    # class_names = train_ds.class_names    # print(f'Classes encontradas: {class_names}')    return train_ds, val_ds, test_ds, class_names

## Construção da Arquitetura do ModeloA função `build_model` define a arquitetura da CNN. O modelo é composto por:- Uma camada de `Rescaling` para normalizar os valores dos pixels.- Três blocos de `Conv2D` + `MaxPooling2D` para extração de características.- Camadas `Flatten`, `Dense` e `Dropout` para a classificação final.O modelo é compilado com o otimizador Adam e a função de perda `sparse_categorical_crossentropy`.

In [9]:
def build_model():    '''Constrói e compila o modelo da Rede Neural Convolucional (CNN).    A arquitetura consiste em três blocos convolucionais seguidos por uma camada densa    de classificação. Uma camada de dropout é usada para regularização.    Retorna:        tf.keras.Model: O modelo de CNN compilado.    '''    model = models.Sequential([        # Camada de normalização para reescalar os pixels para [0,1]        layers.Rescaling(1./255, input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)),        # Camada Convolucional 1        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),        layers.MaxPooling2D((2, 2)),        # Camada Convolucional 2        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),        layers.MaxPooling2D((2, 2)),        # Camada Convolucional 3        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),        layers.MaxPooling2D((2, 2)),        # Camada de Classificação        layers.Flatten(),        layers.Dense(256, activation='relu'),        layers.Dropout(0.5),        layers.Dense(NUM_CLASSES, activation='softmax') # Camada de saída    ])    model.compile(optimizer='adam',                  loss='sparse_categorical_crossentropy',                  metrics=['accuracy'])    return model

## Visualização do Histórico de TreinamentoA função `plot_history` gera gráficos que mostram a evolução da acurácia e da perda nos conjuntos de treino e validação ao longo das épocas. Isso é útil para diagnosticar problemas como overfitting.

In [10]:
def plot_history(history):    '''Plota os gráficos de acurácia e perda do treinamento e validação.    Args:        history (tf.keras.callbacks.History): O objeto de histórico retornado pelo            método `model.fit()`    '''    acc = history.history['accuracy']    val_acc = history.history['val_accuracy']    loss = history.history['loss']    val_loss = history.history['val_loss']    epochs_range = range(len(acc))    plt.figure(figsize=(12, 5))    plt.subplot(1, 2, 1)    plt.plot(epochs_range, acc, label='Acurácia de Treino')    plt.plot(epochs_range, val_acc, label='Acurácia de Validação')    plt.legend(loc='lower right')    plt.title('Acurácia de Treino e Validação')    plt.subplot(1, 2, 2)    plt.plot(epochs_range, loss, label='Perda de Treino')    plt.plot(epochs_range, val_loss, label='Perda de Validação')    plt.legend(loc='upper right')    plt.title('Perda de Treino e Validação')    plt.show()

## Execução PrincipalEsta célula orquestra todo o processo: carrega os dados, otimiza o pipeline de entrada com `cache()` e `prefetch()`, constrói o modelo, treina-o, salva o modelo final e avalia seu desempenho no conjunto de teste. Por fim, exibe os gráficos de treinamento.

In [12]:
if not os.path.exists(DATASET_DIR):    print(f"ERRO: Diretório '{DATASET_DIR}' não encontrado. Execute o script 'gerador_dataset.py' primeiro.")else:    train_ds, val_ds, test_ds, class_names = load_datasets()    # Otimiza a performance de carregamento dos dados    AUTOTUNE = tf.data.AUTOTUNE    train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)    with tf.device('/GPU:0'):        model = build_model()    model.summary()    print("Iniciando o treinamento...")    history = model.fit(        train_ds,        validation_data=val_ds,        epochs=EPOCHS    )    print("Treinamento concluído!")    # Salva o modelo treinado    model.save(MODEL_SAVE_PATH)    print(f"Modelo salvo em '{MODEL_SAVE_PATH}'")    # Avalia o modelo no conjunto de teste    print("Avaliando o modelo no conjunto de teste...")    test_loss, test_acc = model.evaluate(test_ds, verbose=2)    print(f"Acurácia no conjunto de teste: {test_acc:.4f}")    # Plota os gráficos    plot_history(history)

## Exportação dos Nomes das ClassesSalva a lista de nomes de classes em um arquivo JSON. Isso é crucial para que o mesmo mapeamento de índice para classe possa ser usado posteriormente durante a etapa de predição.

In [13]:
# Exportar class_namesimport jsonCLASS_NAMES_PATH = DATA_DIR / raw / class_names.jsonwith open(CLASS_NAMES_PATH, 'w', encoding='utf-8') as f:    json.dump(class_names, f, ensure_ascii=False)print(Nomes das classes salvos em 'class_names.json')print(Processo concluído!)

In [14]:
for idx, name in enumerate(class_names):    print(idx, name)